# Communities and Crime Cross-Validation
Data is taken from [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime).
> U. S. Department of Commerce, Bureau of the Census, Census Of Population And Housing 1990 United States: Summary Tape File 1a & 3a (Computer Files),
>
> U.S. Department Of Commerce, Bureau Of The Census Producer, Washington, DC and Inter-university Consortium for Political and Social Research Ann Arbor, Michigan. (1992)
>
> U.S. Department of Justice, Bureau of Justice Statistics, Law Enforcement Management And Administrative Statistics (Computer File) U.S. Department Of Commerce, Bureau Of The Census Producer, Washington, DC and Inter-university Consortium for Political and Social Research Ann Arbor, Michigan. (1992)
>
> U.S. Department of Justice, Federal Bureau of Investigation, Crime in the United States (Computer File) (1995)
>
> Redmond, M. A. and A. Baveja: A Data-Driven Software Tool for Enabling Cooperative Information Sharing Among Police Departments. European Journal of Operational Research 141 (2002) 660-678.

*This notebook compares the performance of warped linear regression to ordinary least squares on a cross-validation of the data.*

## Import Dependencies

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
np.random.seed(1)

## Load Data

In [2]:
def drop_missing(df):
    drop_columns = []
    for column in df.columns:
        if '?' in list(df[column].values):
            drop_columns.append(column)
    return df.drop(columns=drop_columns)

In [3]:
def load_dataset(filename):
    df = pd.read_csv(filename, header=None)
    df = df.drop(range(5), axis=1)
    df = drop_missing(df)
    X = np.array(df.iloc[:,:-1].values, dtype=float)
    y = np.array(df.iloc[:,-1].values, dtype=float)
    return X, y

In [13]:
X, y = load_dataset("communities.data")
X = StandardScaler().fit_transform(X)

## Set up Cross-Validation Splits

In [7]:
cv_splits = list(KFold(10, shuffle=True).split(X))

## Set Warping Parameters
The warping parameters were fit to maximize the log-likelihood of the training data for each cross-validation using a second order optimizer.

In [8]:
warping_parameters = np.array([
    [29.11410629,  8.2121922 ,  0.09806633],
    [29.5381302 ,  8.18131267,  0.09972036],
    [31.86772755,  8.33747215,  0.09740263],
    [33.0122389 ,  8.22343264,  0.09973035],
    [28.2135524 ,  8.24779062,  0.09725041],
    [29.95950517,  8.11544552,  0.10044195],
    [25.12550611,  8.34099059,  0.09358942],
    [-12.96473319, 8.17310442,   0.0867982],
    [29.97945979,  8.16349394,  0.09975204],
    [35.60438079,  8.34048291,  0.09914208]
])

## Decorator for Vector Functions

In [9]:
def vectorizable(f):
    def f_(x):
        if hasattr(x, '__iter__'):
            return np.array([f(xi) for xi in x])
        return f(x)
    return f_

## Build Warping Function

In [10]:
def make_warper(parameters, y):
    y_mean = np.mean(y)
    y_std = np.std(y)
    def f(t):
        t = (t - y_mean) / (y_std * np.sqrt(len(y)))
        result = t
        for i in range(0, len(parameters), 3):
            a, b, c = parameters[i:(i+3)]
            result += a**2*math.tanh(b**2*(t + c))
        return result
    mean = np.mean([f(yi) for yi in y])
    @vectorizable
    def f_(t):
        return f(t) - mean
    @vectorizable
    def fp(t):
        t = (t - y_mean) / (y_std * np.sqrt(len(y)))
        result = 1.0
        for i in range(0, len(parameters), 3):
            a, b, c = parameters[i:(i+3)]
            u = math.tanh(b**2*(t + c))
            result += a**2*b**2*(1 - u**2)
        result /= y_std * np.sqrt(len(y))
        return result
    return f_, fp

## Compute Log-Likelihood Proxy of Training Data

In [11]:
def compute_log_likelihood_proxy(X, y, phi):
    f, fp = make_warper(phi, y)
    z = f(y)
    model = LinearRegression(fit_intercept=False)
    model.fit(X, z)
    z_pred = model.predict(X)
    rss = sum((z-z_pred)**2)
    return -len(y)/2*np.log(rss) + sum(np.log(fp(y)))

## Verify Maximums
Tweak the warping parameters to verify they're a local maximum of the log-likelihood.

In [12]:
def verify_log_likelihood_opt(X, y, phi):
    delta_x = 1.0e-3
    for i, phi_i in enumerate(phi):
        def f(x):
            phi_copy = np.array(phi)
            phi_copy[i] = x
            return compute_log_likelihood_proxy(X, y, phi_copy)
        f0 = f(phi_i)
        for x in [phi_i - delta_x, phi_i + delta_x]:
            delta_f = f(x) - f0
            relative_delta_f = delta_f / delta_x
            if relative_delta_f > 0 and np.abs(relative_delta_f) > 1.0e-3:
                print(i, x, "\t", delta_f, relative_delta_f)
                assert False, "Can't verify optimum"

In [16]:
for index, (train_indexes, _) in enumerate(cv_splits):
    print(index)
    X_train = X[train_indexes, :]
    y_train = y[train_indexes]
    verify_log_likelihood_opt(X_train, y_train, warping_parameters[index])

0
1
0 29.5371302 	 2.3903871806396637e-06 0.0023903871806396637


AssertionError: Can't verify optimum